In [4]:
import numpy as np
#import time
#import matplotlib.pyplot as plt

from ngsolve import *

from ngsolve.meshes import *
#from ngsolve.solvers import *
#from netgen.geom2d import unit_square



#from su2_yangmills import *
from lgt import *

from ngsolve.webgui import Draw

In [5]:
from pathlib import Path
from ngsolve.fem import CompilePythonModule

lgtcpp_path = "../../src/gaugelinkfe/lgt.cpp" 

if True:
    print("compile it")
    m = CompilePythonModule(Path(lgtcpp_path), init_function_name="lgt_module")
else:
    print("load it")
    #import lgt_module as m

compile it


In file included from /home/sriney/fem/liegroupfem/demos/lgt/../../src/gaugelinkfe/lgt.cpp:9:
/home/sriney/fem/liegroupfem/demos/lgt/../../src/gaugelinkfe/MeshVars.hpp: In member function ‘void MeshVars::CalctBoneMass()’:
/home/sriney/fem/liegroupfem/demos/lgt/../../src/gaugelinkfe/MeshVars.hpp:263:58: error: conversion from ‘ngla::BaseMatrix’ to non-scalar type ‘ngla::SparseMatrix<double>’ requested
  263 |     SparseMatrix<double> tbonemassmat = massbf->GetMatrix();
      |                                         ~~~~~~~~~~~~~~~~~^~


NgException: problem calling compiler

In [ ]:
ne=10
#mesh = MakeStructured2DMesh(quads=True, nx=ne, ny=ne, periodic_x=True, periodic_y=True)
mesh = MakeStructured2DMesh(quads=True, nx=ne, ny=ne, periodic_x=False, periodic_y=False)
#mesh = MakeStructured2DMesh(quads=False, nx=ne, ny=ne, periodic_x=False, periodic_y=False)

#mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))

#mesh = unit_square.GenerateMesh(maxh=0.1)

Draw(mesh)
dt = 0.01
meshvars = m.MeshVars(mesh, dt)

In [ ]:
#draw_tbone_weights(meshvars)
#draw_bone_weights(meshvars)

In [ ]:
fesf = HCurl(mesh, order=1)**3
CFf = CF( (x,y,2*x,-2*y,x+y,x*y) )

#fesf = H1(mesh, order=1)
#CFf = CF( x+y )

gff = GridFunction(fesf)

gff.Set(CFf)

Elnum = 50

m.Evaluatef(meshvars, gff, Elnum)

In [ ]:
### TODO: interpolate continuous A with discrete Us
fesA = HCurl(mesh, order=5)**3

ElNum = 0
ENum = 1

h=0.05

f = 1/h * ((x-h)/h)**4

ElCx = (2*ElNum + 1)*h
xref = (x-ElCx)/h
f = 1/h * (xref)**1
#f = 1/h * sin(2*pi*xref)

#CFA = CF( (0,0, 0,0 ,1,0) )
#CFA = CF( (0,0, 0,0 ,f,0) )
CFA = CF( (0,0, f,0 ,0,0) )

gfA = GridFunction(fesA)

gfA.Set(CFA)

#A = m.IntegratedualA(meshvars, gfA, 21, 0)
A = m.IntegratedualA(meshvars, gfA, ENum, ElNum)
print(A[2])

U = m.SU2(A[0],A[1],A[2], True)

dir(U)
print(U)

In [ ]:
#Utest = m.SU2(2,10,1,3.333)
#print(Utest)

Utest = m.SU2(10,1,3.333)
print(Utest)

In [ ]:
print('output:')
for c in [0,1,2]:
    print(A[c])

In [ ]:
order=0
#fesC = L2(mesh, order=order)**3 # for su2 charge
#fesJ = HDiv(mesh, order=order, discontinuous=True)**3 # for su2 charge current

# discrete connection one form sitting on borders
#fesA = FacetFESpace(mesh, order=0)**3

# parallel transport maps between neighbouring volumes, belonging to an oriented interface
#TODO: orientation?
fesU = FacetFESpace(mesh, order=0)**4


n=1
fesA = HCurl(mesh, order=10)**3

# space of wilson loops centered around bones (= corners (2D) or edges (3D))
# for visualization only!
fesW = H1(mesh, order=order)**4


# space of magnetic energy
fesHB = H1(mesh, order=order)

# space of electric energy
fesHE = HCurl(mesh, order=order)

# space of current flux
fesjflux = FacetFESpace(mesh, order=0)**3

In [ ]:
mu = [0.5, 0.5]
sigma2 = 0.1
r_u =  [x - mu[0], y - mu[1]]
rot_u =  [y - mu[1],x - mu[0]]

def gaussxy(mu, sigma2):
    return exp(-0.5*((x-mu[0])*(x-mu[0]) + (y-mu[1])*(y-mu[1]))/sigma2)


#rotCFA = CF( tuple( rot_u ) )* gaussxy(mu, sigma2)
#CFA = CF( tuple([*rotCFA, 0,0 ,0,0]) )

#CFA = CF( (rot_u[0], rot_u[1], 0,0 ,0,0) )
A0 = 10

def sinp_f(f):
    return sin(0.5*pi*f)


def mexp(f):
    return exp(-1.*f)

sinpr = sin(0.5*pi*( r_u[0]**2 + r_u[1]**2 ))

sinpy =  sin(0.5*pi*(y-0.5))
sinpx = sin(0.5*pi*(x-0.5))

for i in range(10):
    sinpy = sinp_f(sinpy)
    sinpx = sinp_f(sinpx)
    sinpr = sinp_f(sinpr)


signx = IfPos(x - mu[0], 1, -1)
signy = IfPos(y - mu[1], 1, -1)

posy = IfPos(y - mu[1], 1, 0)
posx = IfPos(x - mu[0], 1, 0)

CFA = CF( (A0*rot_u[0]*sinpr, A0*rot_u[1]*sinpr, 0,0 ,0,0) )

#CFA = CF( (A0*posy*posx,0,0,0 ,0,0) )
#CFA = CF( (0,A0*posy*posx,0,0 ,0,0) )

#CFA = CF( (A0*sin(0.5*pi*sin(0.5*pi*sin(0.5*pi*(y-0.5)))),0,0,0 ,0,0) )
#CFA = CF( (A0*sin(pi*sin(pi*sin(pi*y))),0,0,0 ,0,0) )


#CFA = CF( ( rot_u[0], rot_u[1],0,0 ,0,0) )

CFA = CF( (A0*posy,0, 0,0 ,0,0) ) + CF( (-1.*A0*(1.-posy),0, 0,0 ,0,0) )


gfA = GridFunction(fesA)

gfA.Set(CFA)

#Draw(CFA, mesh)
Draw(Norm(gfA), mesh)

gfU = GridFunction(fesU)
m.InterpolateAtoU(meshvars, gfA, gfU)

Enum = 5
U = m.GetLink(gfU, Enum, True)
print(f"U[{Enum}]")
print(U)

gfHB = GridFunction(fesHB)
m.CalcgfHB(meshvars,gfU, gfHB)
Draw(gfHB, mesh)

In [ ]:
fesh1mass = fesh1.Mass()
fesh1massmat = fesh1mass
#help(fesh1mass)
gfuh1 = GridFunction(fesh1)
gfuh1vec = gfuh1.vec
fesh1mass.Mult(gfuh1vec, gfuh1vec)

In [ ]:
meshvars.GettBoneWeight(0)